In [7]:
from bs4 import BeautifulSoup
import pandas as pd
from typing import List

from selenium import webdriver


In [5]:

def cooking_soup(url: str) -> BeautifulSoup:
   """
   This function loads a webpage and all the dynamical elements. It then retrieves the html file as a Beautifulsoup class object
   """
   driver = webdriver.Firefox()
   driver.implicitly_wait(3)
   driver.get(url)
   soup_file = driver.page_source
   soup = BeautifulSoup(soup_file)
   driver.close()
   return soup


def url_getter(soup: BeautifulSoup) -> List[str]:
   """
   This function retrieves all urls of all houses on a page on Immoweb.

   :params soup: BeautifulSoup

   :return: List of str of urls
   """
   list_of_urls = []
   first_tags = soup.find_all('li', attrs={'class': 'search-results__item'})
   houses = []
   for tag in first_tags:
      houses += tag.find_all('a', attrs={'class': 'card__title-link'})
   for house in houses:
      list_of_urls.append(house.get('href'))
   return list_of_urls

In [18]:
def get_next_page(soup: BeautifulSoup) -> str:
    """
    Function that returns the url of the 'next' button on a page and none if there is no 'next' button

    :param soup:  a Beautifulsoup class object
    
    :return: url as a string if there is a 'next' button, if not it returns None
    """
    button = soup.find('ul', attrs={'class': 'pagination'})
    ref = button.find('a', 'pagination__link pagination__link--next button button--text button--size-small')
    if ref:
        return ref.get('href')
    else:
        return None
    
    

In [39]:
# list of subtypes, usefull for threading
sub_types = ['bungalow', 'castle', 'country-cottage', 'apartment-block', 'town-house', 'chalet', 'farmhouse', 'exceptional-property','mixed-use-building', 'mansion', 'villa', 'manor-house','pavilion', 'other-property', 'duplex', 'triplex', 'studio', 'penthouse', 'loft', 'flat-studio']

In [67]:
def get_all_urls(name: str):
    """
    Function that gets the name of a subtype of property on Immoweb and makes a csv file of all urls of all properties of that subtype on Immoweb

    :param name: name as str
    """
    all_url = []
    new_url = f"https://www.immoweb.be/en/search/{name}/for-sale?countries=BE&page=1&orderBy=relevance"
    while new_url and (len(all_url)<5000):
        soup = cooking_soup(new_url)
        new_url = get_next_page(soup)
        list = url_getter(soup)
        all_url += list
    url_list = pd.Series(all_url)
    url_list.to_csv(f"urls_{name}")

In [68]:
from threading import Thread

class urlScrapper(Thread):
    """
    A class to speed up looking for urls of properties on Immoweb

    Attributes
    ----------

    : param name is a str representing the subtype of propertie we want to retrieve from Immoweb
    """
    def __init__(self, name):
        Thread.__init__(self)
        self.name = name
    
    def run(self):
        get_all_urls(self.name)

In [82]:
# We make a dictionary of Threads. Experience learns that 6 Threads at once is a good comprimise.
dict = {}
for name in sub_types[16:]:
    dict[name] = urlScrapper(name)

In [83]:
# We run each thread
for name in sub_types[16:]:
    dict[name].start()

Exception in thread studio:
Traceback (most recent call last):
  File "/Users/wvdvijve/opt/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "<ipython-input-68-450049ff4bac>", line 10, in run
  File "<ipython-input-67-0ceea70cd126>", line 5, in get_all_urls
  File "<ipython-input-5-511eb66763b3>", line 9, in cooking_soup
  File "/Users/wvdvijve/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 926, in page_source
    return self.execute(Command.GET_PAGE_SOURCE)['value']
  File "/Users/wvdvijve/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py", line 424, in execute
    self.error_handler.check_response(response)
  File "/Users/wvdvijve/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/errorhandler.py", line 247, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchWindowException: Message: Browsing context h